In [1]:
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
MOUNT_DIR = '/content/drive/MyDrive/BarclaysReviews/'
DATA_DIR = '/content/drive/MyDrive/BarclaysReviews/data/'
MODEL_DIR = '/content/drive/MyDrive/BarclaysReviews/model/'


In [6]:
final_merged_reviews = pd.read_csv(DATA_DIR+"final_merged_reviews.csv")

In [8]:
ratingColName = 'customer_sentiment'
final_merged_reviews[ratingColName] = 'POSITIVE'
for i in final_merged_reviews.index:
    if final_merged_reviews['rating'][i] > 3:
      final_merged_reviews[ratingColName][i] = 'POSITIVE'
    else:
      final_merged_reviews[ratingColName][i] = 'NEGATIVE'

<ipython-input-8-c81c8ff3faa7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_reviews[ratingColName][i] = 'NEGATIVE'
<ipython-input-8-c81c8ff3faa7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_reviews[ratingColName][i] = 'POSITIVE'


In [42]:
import spacy 
nlp = spacy.load('en_core_web_sm')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
barclays_stop_list = ['app', 'barclays','need','will','pay','work']
stop_words.extend(barclays_stop_list)
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [43]:
final_merged_reviews_not_null = final_merged_reviews[final_merged_reviews['review_description'].notnull() & final_merged_reviews[ratingColName].str.contains('NEGATIVE')]
replace_dict = {"google pay":"google","contactless service":"contactless","credit card":"credit","debit card":"debit",''}
final_merged_reviews_not_null['review_description'].replace(replace_dict,inplace=True,regex=True)
#work , pay -cheque = cheque , ok, calender view = calender 
#final_merged_reviews_not_null[final_merged_reviews_not_null["review_description"].str.contains("google")][["review_description", "rating",ratingColName]] 

In [13]:
final_merged_reviews_not_null[final_merged_reviews_not_null["review_description"].str.contains("contactles")][["review_description", "rating",ratingColName]].shape

(726, 3)

In [14]:

import re
wnl = WordNetLemmatizer()
def tokenizer(text):
    regex_num_ponctuation = '(\d+)|([^\w\s])'
    regex_little_words = r'(\b\w{1,2}\b)'
    return [wnl.lemmatize(token.lemma_.lower()) for token in nlp(text) if not token.is_stop and not token.is_punct ]

def LemmaTokenizer(doc):
      wnl = WordNetLemmatizer()
      regex_num_ponctuation = '(\d+)|([^\w\s])'
      regex_little_words = r'(\b\w{1,2}\b)'
      return [wnl.lemmatize(t) for t in word_tokenize(doc) 
              if not re.search(regex_num_ponctuation, t) and not re.search(regex_little_words, t) and not t.is_stop and not t.is_punct]

In [15]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.9/320.9 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 14.8 MB/s eta 0:00:00
  Created wheel for lda: filename=lda-2.0.0-cp310-cp310-linux_x86_64.whl size=628188 sha256=f8ccde77d30dad51320b365defbd026355e364b930d7e8cb892d9ee6c3025578
  Stored in directory: /root/.cache/pip/wheels/9d/32/fa/07fdf278a0b24269ce4d0be3d642c28dd0e888a4c5f2031279
Successfully built lda


In [16]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=4ccc2b4e07d7bcafd4f8fef3db71e2afca69d2d97346a1a97e458165857f6e41
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55640 sha256=d35fc41e32647e44c446b4417ad085bbc2ee43edd8f6b715ff846057a200af87
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [17]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=c8ea3341ebf68090dba70953959f9490d43778d7a7588f10245a20f0304277d2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [18]:
!pip install HDBSCAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for HDBSCAN: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=3541526 sha256=b6121645818a7921fa4a7bdc8fac7d35de04fd827223aa472f66cc3bb943398f
  Stored in directory: /root/.cache/pip/wheels/dc/52/e3/6c6b60b126b4d5c4370cb5ac071b82950f91649d62d72f7f56
Successfully built HDBSCAN


In [21]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 11.9 MB/s eta 0:00:00


In [19]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer 

In [22]:
from bertopic import BERTopic

In [23]:
import time

In [49]:
def topicModellingBERT(df, textCol = "review_description",sentiment_col = "",sentiment="",min_df=4,max_features = 10000,top_n_words=5,path="",model_name=""):
  min_df = min_df # minimum required occurences of a word, e.g 4
  max_features = max_features # max number of unique words, e.g 10000

  vectorizer_model = CountVectorizer(
                min_df=min_df, 
                max_features=max_features, 
                tokenizer=tokenizer, # use tokenizer we defined before
                ngram_range=(1,2),
                stop_words=stop_words) # we use unigram & bigram
  # X = vectorizer.fit_transform(final_merged_reviews_not_null['review_description'])

  if(sentiment!=""):
      df = df[df[textCol].notnull() & df[sentiment_col].str.contains(sentiment)]
      print(sentiment,'Topic Modelling on : ',sentiment_col,'column')
  print('dataet length: ',df.shape,'\n')
  # # we add this to remove stopwords that can pollute topcs
  vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stop_words)
  df.reset_index(drop=True, inplace=True)
  model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
  )
  topics, probs = model.fit_transform(df[textCol])
  print(model.get_topic_info())
  
  model.save(path+model_name+'_'+str(int(time.time())))
  return  topics, probs, model



In [56]:
 topics, probs, model = topicModellingBERT(df=final_merged_reviews,sentiment_col="customer_sentiment",sentiment='NEGATIVE',path=MODEL_DIR,model_name='negative_topic_modelling')
 topics

NEGATIVE Topic Modelling on :  customer_sentiment column
dataet length:  (5746, 17) 



Batches:   0%|          | 0/180 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [53]:
model.visualize_barchart(top_n_topics=10)


In [54]:
model.visualize_barchart(topics=[-1])

In [41]:
model.visualize_barchart(topics=[-1])

In [63]:
topics

[0,
 0,
 -1,
 0,
 2,
 0,
 2,
 0,
 9,
 0,
 5,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 9,
 1,
 1,
 -1,
 0,
 0,
 0,
 2,
 1,
 6,
 0,
 0,
 3,
 2,
 0,
 3,
 9,
 0,
 1,
 7,
 -1,
 1,
 3,
 5,
 1,
 1,
 5,
 2,
 1,
 1,
 1,
 2,
 0,
 6,
 2,
 1,
 0,
 3,
 0,
 0,
 0,
 0,
 6,
 0,
 2,
 0,
 0,
 8,
 1,
 8,
 2,
 1,
 0,
 1,
 1,
 0,
 0,
 8,
 0,
 2,
 2,
 2,
 1,
 3,
 0,
 0,
 0,
 3,
 2,
 0,
 3,
 2,
 3,
 6,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 -1,
 3,
 3,
 2,
 1,
 3,
 2,
 0,
 -1,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 6,
 3,
 1,
 0,
 0,
 4,
 3,
 1,
 2,
 0,
 0,
 2,
 2,
 8,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 1,
 0,
 2,
 2,
 8,
 0,
 1,
 2,
 3,
 6,
 1,
 2,
 0,
 4,
 9,
 0,
 0,
 8,
 1,
 0,
 0,
 2,
 -1,
 0,
 4,
 1,
 2,
 8,
 9,
 -1,
 2,
 0,
 -1,
 8,
 4,
 1,
 0,
 1,
 1,
 2,
 2,
 0,
 1,
 1,
 -1,
 1,
 1,
 8,
 2,
 1,
 3,
 1,
 3,
 2,
 0,
 3,
 2,
 0,
 0,
 2,
 0,
 8,
 1,
 2,
 3,
 2,
 1,
 6,
 2,
 9,
 6,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 2,
 8,
 2,
 2,
 0,
 8,
 2,
 2,
 0,
 0,
 0,
 -1,
 4,
 4,
 8,
 6,
 8,
 1,
 2

In [64]:
 import pickle

In [66]:
with open(MODEL_DIR+'topic_negative.pkl', 'wb') as f:
  pickle.dump(topics, f)

In [ ]:
# with open('parrot.pkl', 'rb') as f:
# ...   mynewlist = pickle.load(f)